## Data Analysis

In [1]:
import os
import dotenv

dotenv.load_dotenv()
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

conString = {'host':DB_HOST,
             'user':DB_USER,
             'password':DB_PASS,
             'port':DB_PORT}

# Data manipulation
import pandas as pd
import numpy as np
import psycopg2

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualizations
import altair as alt
import matplotlib as plt
alt.data_transformers.disable_max_rows()  # disable error for big datasets